In [ ]:
# pipenv install pandas scipy plotly scikit-learn optuna shap ipywidgets nbformat numpy==2.0

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import plot_tree

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, log_loss, roc_curve, roc_auc_score, f1_score, precision_score, recall_score

import optuna
import shap

In [ ]:
pd.__version__ 

In [ ]:
df = pd.read_csv('dataset.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df['data_contratacao'] = pd.to_datetime(df['data_contratacao'], format="%Y-%m-%d")
df['data_demissao'] = pd.to_datetime(df['data_demissao'], format='%Y-%m-%d')
df['data_ultimo_feedback'] = pd.to_datetime(df['data_ultimo_feedback'], format='%Y-%m-%d')
df['data_ultimo_aumento'] = pd.to_datetime(df['data_ultimo_aumento'], format='%Y-%m-%d')
df['data_ultima_mudanca_cargo'] = pd.to_datetime(df['data_ultima_mudanca_cargo'], format='%Y-%m-%d')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df[df['data_contratacao'] == '2020-01-02']

In [ ]:
df.describe()

In [ ]:
df['tempo_empresa'] = df.apply(lambda x: (x['data_demissao'] - x['data_contratacao']).days if x['churn'] == 1 else (pd.Timestamp.now() - x['data_contratacao']).days, axis=1)

In [ ]:
df.head()

In [ ]:
df[df['churn'] == 1].head()

In [ ]:
df['tempo_desde_ultimo_feedback'] = df.apply(lambda x: (pd.Timestamp.now() - x['data_ultimo_feedback']).days, axis=1)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df['dias_desde_ultimo_aumento'] = df.apply(lambda x: (pd.Timestamp.now() - x['data_ultimo_aumento']).days, axis=1)

In [ ]:
df['dias_desde_ultima_mudanca_cargo'] = df.apply(lambda x: (pd.Timestamp.now() - x['data_ultima_mudanca_cargo']).days, axis=1)

In [ ]:
df.tail()

In [ ]:
df.drop(columns=['id'], inplace=True, axis=1)

In [ ]:
df.head()

### EDA

In [ ]:
df.isnull().sum()

In [ ]:
df['churn'].value_counts()

In [ ]:
fig = px.bar(df['churn'].value_counts() / len(df) * 100, title='Fator de Churn')
fig.show()

In [ ]:
for col in df.select_dtypes(include=['object']).columns:
  print(f"Valores únicos na coluna {col}:", df[col].unique())

In [ ]:
df.select_dtypes(include=['int64', 'float64']).describe()

In [ ]:
for col in df.select_dtypes(include=['int64', 'float64']).columns:
  if col != 'churn':
    fig = px.box(df, x='churn', y=col, title=f'Boxplot da coluna {col}', color='churn')
    fig.show()

In [ ]:
colunas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
corr_matrix = df[colunas_numericas].corr()

fig = px.imshow(corr_matrix, title='Matriz de Correlação', color_continuous_scale='Viridis', zmin=-1, zmax=1)

fig.update_traces(text=corr_matrix, texttemplate='%{text:.1%}', textfont={'size': 12})
fig.update_layout(width=1000, height=800, title_font=dict(size=14), font=dict(size=10))

fig.show()

In [ ]:
fig = px.scatter_matrix(df, dimensions=colunas_numericas, color='churn', title='Scatter Matrix')
fig.update_layout(width=1200, height=1000, title_font=dict(size=14), font=dict(size=10))

fig.show()

In [ ]:
colunas_categoricas = df.select_dtypes(include=['object']).columns
for col in colunas_categoricas:
  contigency_table = pd.crosstab(df[col], df['churn'])
  chi2, p_value, dof, expected = chi2_contingency(contigency_table)
  print(f'\nTeste Chi-quadrado para {col} vs Churn')
  print(f'p-value: {p_value}')
  if p_value <= 0.05:
    print('Variáveis são dependentes')
  else:
    print('Variáveis são independentes')

In [ ]:
for col in colunas_categoricas:
  fig = px.histogram(df, x=col, color='churn', title=f'Histograma da coluna {col}', barmode='group')
  fig.show()

### PREPARACAO DOS DADOS

In [ ]:
colunas_datas = df.select_dtypes(include=['datetime64']).columns
df.drop(columns=colunas_datas, inplace=True)

In [ ]:
df.info()

In [ ]:
X = df.drop(columns=['churn', 'tipo_demissao'])
y = df['churn']

In [ ]:
features_numericas =  X.select_dtypes(include=['int64', 'float64']).columns
features_categoricas = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
  transformers=[
    ('num', StandardScaler(), features_numericas),
    ('cat', OneHotEncoder(handle_unknown='ignore'), features_categoricas)
  ]
)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=True, random_state=42) 

In [ ]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
print(X_train.shape, X_test.shape)  

### TREINAMENTO DO MODELO BASELINE

In [ ]:
rf_model = RandomForestClassifier(
  n_estimators=100, 
  max_depth=20, 
  min_samples_split=2, 
  min_samples_leaf=1, 
  random_state=42,
  max_features='sqrt',
  class_weight='balanced'
)

In [ ]:
rf_model.fit(X_train, y_train)

### ANÁLISE DOS RESULTADOS BASELINE

In [ ]:
y_pred = rf_model.predict(X_test)

In [ ]:
len(y_pred)

In [ ]:
y_pred_proba = rf_model.predict_proba(X_test)

In [ ]:
y_pred_proba

In [ ]:
print(classification_report(y_test, y_pred))  

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba[:, 1])

In [ ]:
roc_auc = roc_auc_score(y_test, y_pred_proba[:, 1])

In [ ]:
fig = px.area(
  x=fpr,
  y=tpr,
  title=f'Curva ROC (AUC={roc_auc:.4f})',
  labels=dict(x='Taxa de Falso Positivo', y='Taxa de Verdadeiro Positivo'),
  width=800,
  height=800
)
fig.add_shape(
  type='line', line=dict(dash='dash'),
  x0=0, x1=1, y0=0, y1=1 
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')

fig.show()

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
fig = ConfusionMatrixDisplay(conf_matrix, display_labels=['Não Churn', 'Churn'])
fig.plot()

In [ ]:
print(f'Log Loss: {log_loss(y_test, y_pred_proba)}')

### TREINAR MODELO CROSS VALIDATION E TUNING DE HIPERPARAMETROS

In [ ]:
# Dicionario de hiperparâmetros
params_grids = {
  'n_estimators': [100, 200, 300, 400, 500],
  'max_depth': [None, 10, 20, 30, 40, 50],
  'min_samples_split': [2, 5, 10, 20],
  'min_samples_leaf': [1, 2, 4, 5, 10],
}

In [ ]:
rf_model_cv = RandomForestClassifier(random_state=42, class_weight='balanced', max_features='sqrt')
k_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(rf_model_cv, param_grid=params_grids, cv=k_folds, scoring='recall', verbose=2)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
best_params = grid_search.best_params_

In [ ]:
print(best_params)

In [ ]:
best_score = grid_search.best_score_
best_score

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
y_ped_proba = best_model.predict_proba(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba[:, 1])

In [ ]:
roc_auc = roc_auc_score(y_test, y_pred_proba[:, 1])

In [ ]:
fig = px.area(
  x=fpr,
  y=tpr,
  title=f'Curva ROC (AUC={roc_auc:.4f})',
  labels=dict(x='Taxa de Falso Positivo', y='Taxa de Verdadeiro Positivo'),
  width=800,
  height=800
)
fig.add_shape(
  type='line', line=dict(dash='dash'),
  x0=0, x1=1, y0=0, y1=1 
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')

fig.show()

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
fig = ConfusionMatrixDisplay(conf_matrix, display_labels=['Não Churn', 'Churn'])
fig.plot()

### AJUSTAR THRESHOLD

In [ ]:
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]

recalls = []

for threshold in thresholds:
  y_pred_threshold = (y_pred_proba[:, 1] >= threshold).astype(int)
  recall = recall_score(y_test, y_pred_threshold)
  
  recalls.append(recall)

recalls

In [ ]:
df_trheadholds = pd.DataFrame({'thresholds': thresholds, 'recall': recalls})

In [ ]:
df_trheadholds

In [ ]:
y_pred = (y_pred_proba[:, 1] >= 0.1).astype(int)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba[:, 1])

In [ ]:
roc_auc = roc_auc_score(y_test, y_pred_proba[:, 1])

In [ ]:
fig = px.area(
  x=fpr,
  y=tpr,
  title=f'Curva ROC (AUC={roc_auc:.4f})',
  labels=dict(x='Taxa de Falso Positivo', y='Taxa de Verdadeiro Positivo'),
  width=800,
  height=800
)
fig.add_shape(
  type='line', line=dict(dash='dash'),
  x0=0, x1=1, y0=0, y1=1 
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')

fig.show()

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
fig = ConfusionMatrixDisplay(conf_matrix, display_labels=['Não Churn', 'Churn'])
fig.plot()

### OBTER IMPORTANCIA DAS VARIAVEIS

In [ ]:
importancias = best_model.feature_importances_
nomes_features = preprocessor.get_feature_names_out()

df_importancias = pd.DataFrame({'feature': nomes_features, 'importancia': importancias})

In [ ]:
df_importancias = df_importancias.sort_values(by='importancia', ascending=False).reset_index(drop=True)

In [ ]:
df_importancias

In [ ]:
fig = px.bar(df_importancias.head(10), 
             x='importancia', 
             y='feature', 
             title='Importância das Features', 
             orientation='h', 
             color='importancia', 
             color_continuous_scale='Viridis')
fig.show()

### VISUALIZAR ARVORE

In [ ]:
def visualiza_arvore(modelo, indice_arvore, max_profundidade=5):
  plt.figure(figsize=(20, 20))
  plot_tree(modelo.estimators_[indice_arvore], 
            max_depth=max_profundidade, 
            filled=True, 
            feature_names=nomes_features,
            class_names=['Não Churn', 'Churn'],
            fontsize=9,
            proportion=True,
            precision=2,
            rounded=True)
  plt.title(f'Árvore {indice_arvore} da Random Forest', fontsize=14)
  plt.tight_layout()
  plt.show()

In [ ]:
for i in range(5):
  visualiza_arvore(best_model, i)

### SHAPLEY VALUES

In [ ]:
explainer_class = shap.Explainer(best_model.predict, X_train, feature_names=nomes_features)

In [ ]:
shape_values = explainer_class(X_test)

In [ ]:
shap.plots.bar(shape_values)

In [ ]:
shap.plots.beeswarm(shape_values, max_display=10)

### SHAPLEY VALUE DE UMA LINHA ESPECIFICA

In [ ]:
shap.plots.waterfall(shape_values[1])